In [ ]:
import sys
import itertools
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent))
from os.path import join, basename

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

from bulletin import default_input, default_output, hoje, ontem, tables_path
from bulletin.systems import Notifica, CasosConfirmados
from bulletin.utils import utils, normalize
from bulletin.utils.static import Municipios

import random
import gc
# from tqdm import tqdm
from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

In [ ]:
from bulletin.systems.sim import Sim

sim = Sim()
sim.read_all_database_files()
sim.save('dopr',replace=True,compress=False)
# sim.load('dopr',compress=False)

sim.to_notifica()

sim.df['cod_cbo'] = sim.df['cod_cbo'].apply(lambda x: int(str(x)[:4]))
sim.df.loc[sim.df['cod_cbo']==2231,'cod_cbo'] = 2251

sim_covid = sim.df.loc[sim.df['evolucao']==2].copy()

In [ ]:
sim.dtypes['IDADE']

In [ ]:
sim.df.dtypes

In [ ]:
sim_covid.shape

In [ ]:
nt = Notifica()
nt.load('notifica',compress=False)

In [ ]:
saude = pd.read_pickle(join(default_input, 'outros', 'prof_saude_base.pkl'))
saude.shape

In [ ]:
cod_cbo = nt.tables['cod_cbo'].set_index('id')['value'].rename('cod_cbo')

In [ ]:
ocupacao = nt.tables['ocupacao'].set_index('id')['ocupacao']

In [ ]:
#22 PROFISSIONAIS DAS CIÊNCIAS BIOLÓGICAS, DA SAÚDE E AFINS 
#32 TÉCNICOS DE NÍVEL MÉDIO DAS CIÊNCIAS BIOLÓGICAS, BIOQUÍMICAS, DA SAÚDE E AFINS
cbo_trab_saud = ['223','225','226','251','320','322','324','325','510','515','516'] 
# cbo_trab_saud = ['223','225','251','320','324'] #516 pra pegar cuidador de idosos, mas pegou mta coisa aleatória

# trab_saud =  nt.df.loc[(nt.df['ocupacao'].isin([1,3]) | nt.df['cod_cbo'].astype(str).str[:3].isin(cbo_trab_saud))].copy()
# trab_saud =  nt.df.loc[(nt.df['classificacao_final']==2) & nt.df['cod_cbo'].astype(str).str[:3].isin(cbo_trab_saud)].copy()
trab_saud =  nt.df.loc[nt.df['cod_cbo'].astype(str).str[:3].isin(cbo_trab_saud)].copy()
trab_saud['cbo'] = trab_saud['cod_cbo'].replace(cod_cbo)

trab_saud_obt =  sim_covid.loc[sim_covid['cod_cbo'].astype(str).str[:3].isin(cbo_trab_saud)].copy()
trab_saud_obt['cbo'] = trab_saud_obt['cod_cbo'].replace(cod_cbo)
trab_saud.shape

In [ ]:
trab_saud = pd.concat([trab_saud, saude], ignore_index=True)
trab_saud.shape

In [ ]:
trab_saud = trab_saud.drop_duplicates(subset='id', keep='last')
trab_saud.shape

In [ ]:
trab_saud_obt.groupby('evolucao')[['paciente']].count()

In [ ]:
trab_saud.groupby('evolucao')[['paciente']].count()

In [ ]:
## obitos trabalhadores da saude não evoluido como obito
trab_saud.loc[(trab_saud['evolucao']!=2) & (
    trab_saud.loc[trab_saud['cns'].notna(),'cns'].isin(trab_saud_obt['cns']) |
    trab_saud.loc[trab_saud['hash_mae'].notna(),'hash_mae'].isin(trab_saud_obt['hash_mae']) |
    trab_saud.loc[trab_saud['hash_nasc'].notna(),'hash_nasc'].isin(trab_saud_obt['hash_nasc'])
),'evolucao'] = 2

In [ ]:
trab_saud.groupby('evolucao')[['paciente']].count()

In [ ]:
## obitos trabalhadores da saude não notificados

trab_saud = pd.concat([
    trab_saud,
    trab_saud_obt.loc[~(
        trab_saud_obt.loc[trab_saud_obt['cns'].notna(),'cns'].isin(trab_saud['cns']) |
        trab_saud_obt.loc[trab_saud_obt['hash_mae'].notna(),'hash_mae'].isin(trab_saud['hash_mae']) |
        trab_saud_obt.loc[trab_saud_obt['hash_nasc'].notna(),'hash_nasc'].isin(trab_saud['hash_nasc'])
    )]
])

trab_saud.loc[trab_saud['id'].isna(),'classificacao_final'] = 2
trab_saud.loc[trab_saud['id'].isna(),'data_diagnostico'] = trab_saud.loc[trab_saud['id'].isna(),'data_cura_obito']

trab_saud.loc[trab_saud['metodo']!=1,'metodo'] = 'Teste Rápido/Outro'
trab_saud.loc[trab_saud['metodo']==1,'metodo'] = 'RT-PCR'

In [ ]:
trab_saud.groupby('evolucao')[['paciente']].count()

In [ ]:
# AJUSTES NECESSÁRIOS
trab_saud['uf_residencia'] = trab_saud['uf_residencia'].fillna(41).astype(int)
trab_saud.loc[(trab_saud['evolucao']==3) & ((hoje - trab_saud['data_diagnostico']).dt.days >90),'evolucao'] = 1
trab_saud.loc[trab_saud['evolucao']==2, 'classificacao_final'] = 2



trab_saud.loc[trab_saud['cod_cbo']==0,'cbo'] = 'Não Informado'
trab_saud['cbo'] = trab_saud['cbo'].astype(str).apply(normalize.normalize_text).str.lower()


trab_saud = trab_saud.loc[(trab_saud['uf_residencia'] == 41)]

In [ ]:
trab_saud.loc[trab_saud['cbo'].str.contains('enferm'),'cbo_n'] = 'Enfermagem'
trab_saud.loc[trab_saud['cbo'].str.contains('medic'),'cbo_n'] = 'Médico'
trab_saud.loc[trab_saud['cbo'].str.contains('servic'),'cbo_n'] = 'Serviços'
trab_saud.loc[trab_saud['cbo'].str.contains('administ'),'cbo_n'] = 'Administrativa'
trab_saud.loc[trab_saud['cbo'].str.contains('agentes da saude'),'cbo_n'] = 'Agente Comunitário de Saúde'
trab_saud.loc[trab_saud['cbo'].str.contains('farmac'),'cbo_n'] = 'Área Farmaceutica'
trab_saud.loc[trab_saud['cbo'].str.contains('transpor'),'cbo_n'] = 'Transporte'
trab_saud.loc[trab_saud['cbo'].str.contains('motoris'),'cbo_n'] = 'Transporte'
trab_saud.loc[trab_saud['cbo'].str.contains('dent'),'cbo_n'] = 'Odontologia'
trab_saud.loc[trab_saud['cbo'].str.contains('odon'),'cbo_n'] = 'Odontologia'
trab_saud.loc[trab_saud['cbo'].str.contains('fisio'),'cbo_n'] = 'Fisioterapia'
trab_saud.loc[trab_saud['cbo'].str.contains('lab'),'cbo_n'] = 'Área Laboratorial/Diagnóstico'
trab_saud.loc[trab_saud['cbo'].str.contains('nutri'),'cbo_n'] = 'Alimentação'
trab_saud.loc[trab_saud['cbo'].str.contains('alimentacao'),'cbo_n'] = 'Alimentação'
trab_saud.loc[trab_saud['cbo'].str.contains('veter'),'cbo_n'] = 'Veterinários'
trab_saud.loc[trab_saud['cbo'].str.contains('socorrista'),'cbo_n'] = 'Socorrista'
trab_saud.loc[trab_saud['cbo'].str.contains('servico social'),'cbo_n'] = 'Serviço Social'
trab_saud.loc[trab_saud['cbo'].str.contains('idosos'),'cbo_n'] = 'Cuidador de Idosos'
trab_saud.loc[trab_saud['cbo'].str.contains('fono'),'cbo_n'] = 'Fonoaudiologia'
trab_saud.loc[trab_saud['cbo'].str.contains('psicologos'),'cbo_n'] = 'Psicologia'
trab_saud.loc[trab_saud['cbo'].str.contains('nao informado'),'cbo_n'] = 'Não Informado'

trab_saud['cbo_n'] = trab_saud['cbo_n'].fillna('Outros Trabalhadores')

In [ ]:
trab_saud.groupby('evolucao')[['paciente']].count()

In [ ]:
notificados = trab_saud.groupby(['cbo_n','cbo'])['paciente'].count().sort_values(ascending=False).rename('Notificados')

In [ ]:
metodo = trab_saud.loc[trab_saud['classificacao_final']==2].groupby(['cbo_n','cbo','metodo'])[['paciente']].count().unstack('metodo').fillna(0).astype(int).droplevel(axis=1,level=0)

In [ ]:
confirmados = trab_saud.loc[trab_saud['classificacao_final']==2].groupby(['cbo_n','cbo','evolucao'])[['paciente']].count().unstack('evolucao').fillna(0).astype(int).droplevel(axis=1,level=0)
confirmados['Confirmados'] = confirmados.sum(axis=1)
confirmados = confirmados.rename(columns={1: 'Recuperados',2: 'Óbitos', 3: 'Ativos'})
confirmados = confirmados[['Confirmados','Óbitos','Recuperados','Ativos']]
confirmados = confirmados.sort_values('Confirmados',ascending=False)

In [ ]:
tabela = metodo.join(confirmados).join(notificados).sort_values('Confirmados',ascending=False)

tabela_total = tabela.groupby('cbo_n').sum().sort_values('Confirmados',ascending=False)

# tmp = tabela_total.loc['Outros Trabalhadores'].copy()
# tabela_total = tabela_total.drop(index='Outros Trabalhadores')
# tabela_total = tabela_total.append(tmp)

# tmp = tabela_total.loc['Não Informado'].copy()
# tabela_total = tabela_total.drop(index='Não Informado')
# tabela_total = tabela_total.append(tmp)


tabela_total.loc['Total Geral',:] = tabela.sum(axis=0)
tabela_total = tabela_total.fillna(0).astype(int)

In [ ]:
tabela_total.reset_index(inplace=True)
tabela_total = tabela_total.rename(columns={'cbo_n':'Ocupação'})
tabela_total

In [ ]:
tabela = tabela.sort_index(axis=0,level='cbo_n')
tabela

In [ ]:
ts = Notifica('trab_saud')
ts.df = trab_saud
ts.save(replace=True,compress=False)

In [ ]:
casosPR = ts.df.loc[ts.df['classificacao_final']==2].copy()
casosPR.loc[casosPR['data_diagnostico'].isna(), 'data_diagnostico'] = casosPR.loc[casosPR['data_diagnostico'].isna(), 'data_notificacao']


obitosPR = casosPR.loc[casosPR['evolucao']==2].copy()
obitosPR.loc[obitosPR['data_cura_obito'].isna(), 'data_cura_obito'] = obitosPR.loc[obitosPR['data_cura_obito'].isna(), 'data_diagnostico']


print(len(casosPR))
print(len(obitosPR))

In [ ]:
casosPR['ano_caso'] = casosPR['data_diagnostico'].dt.year
casosPR['mes_caso'] = casosPR['data_diagnostico'].dt.month

obitosPR['ano_obito'] = obitosPR['data_cura_obito'].dt.year
obitosPR['mes_obito'] = obitosPR['data_cura_obito'].dt.month


casos_medicos = casosPR.loc[casosPR['cbo_n'] == 'Médico']
obitos_medicos = obitosPR.loc[obitosPR['cbo_n'] == 'Médico']

In [ ]:
meses = {1:'Janeiro',2:'Fevereiro',3:'Março',4:'Abril',5:'Maio',6:'Junho',7:'Julho',8:'Agosto',9:'Setembro',10:'Outubro',11:'Novembro',12:'Dezembro'}

In [ ]:
writer = pd.ExcelWriter(join(default_output,'profissionais', f"dados_fontes_profissionais.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

In [ ]:
tabela_total.to_excel(writer, 'totais')

In [ ]:
tabela_casos = casosPR.groupby(['ano_caso', 'mes_caso'])[['paciente']].count().rename(columns={'paciente':'quantitativo'})
tabela_casos.rename(index=meses, inplace=True)
tabela_casos.reset_index().to_excel(writer, 'casos_por_mes')

In [ ]:
tabela_casos_medicos = casos_medicos.groupby(['ano_caso', 'mes_caso'])[['paciente']].count().rename(columns={'paciente':'quantitativo'})
tabela_casos_medicos.rename(index=meses, inplace=True)
tabela_casos_medicos.reset_index().to_excel(writer, 'casos_por_mes_medicos')

In [ ]:
tabela_obitos = obitosPR.groupby(['ano_obito', 'mes_obito'])[['paciente']].count().rename(columns={'paciente':'quantitativo'})
tabela_obitos.rename(index=meses, inplace=True)
tabela_obitos.reset_index().to_excel(writer, 'obitos_por_mes')

In [ ]:
tabela_obitos_medicos = obitos_medicos.groupby(['ano_obito', 'mes_obito'])[['paciente']].count().rename(columns={'paciente':'quantitativo'})
tabela_obitos_medicos.rename(index=meses, inplace=True)
tabela_obitos_medicos.reset_index().to_excel(writer, 'obitos_por_mes_medicos')

In [ ]:
writer.save()
writer.close()